# Fast OSM XML parging to igraph
An implementation of XML -> igraph.Graph (directed) parser for OSM files.\
Similar to XML -> networkx.MultiDiGraph parser.

In [18]:
import xml
from dataclasses import dataclass, field
from typing import Optional
from xml.sax import xmlreader
from xml.sax.handler import ContentHandler

import igraph as ig

TAGS_TO_KEEP = {
    "node": {
        "ref",
        "highway",
    },
    "way": {
        "bridge",
        "tunnel",
        "oneway",
        "lanes",
        "ref",
        "name",
        "highway",
        "maxspeed",
        "service",
        "access",
        "area",
        "landuse",
        "width",
        "est_width",
        "junction",
    }
}
# https://wiki.openstreetmap.org/wiki/Key:oneway
ONE_WAY_PATH_OPTIONS = {"yes", "true", "1", "-1", "reverse"}
REVERSED_PATH_OPTIONS = {"-1", "reversed"}

In [19]:
@dataclass(slots=True)
class CoreElementBuffer:
    """Class for storing parsed parameters of XML element (OSM node or way)."""

    type: str  # either "node" or "way"
    osmid: int

    # Parameters to add to node or edges, filled while parsing tag elements
    attrs: dict[str, str | int | float | bool] = field(default_factory=dict, init=False)

    # List of osmids of nodes in a way, filled while parsing nd elements
    path: list[int] = field(default_factory=list, init=False)

In [20]:
class OSMParser(ContentHandler):
    """
    Handler for parsing xml osm file to igraph.Graph (directed).

    Since igraph recalculates is build for static graphs and it recalculates
    indexes every time a changes is introduced - handler first reads to buffers
    and creates graph after the whole file is parsed.

    OSM ids are attributes of nodes in the graph (not actual ids), since igraph
    automatically assigns ids to newly created nodes (ints starting from 0),
    Separate dict is maintained for quick osmid -> igraph id lookups.
    """
    def __init__(self) -> None:
        self.g: Optional[ig.Graph] = None

        # Buffer for currently processed core element
        self._element: Optional[CoreElementBuffer] = None

        # Edges buffer (pairs of osmids)
        self._es: Optional[list[tuple[int, int]]] = None
        # Edge attributes buffer (same length as edges buffer)
        self._es_attrs: Optional[list[dict[str, str | int | float]]] = None
        # Vertex attributes buffer (of length equal to amount of vertices in a graph)
        self._vs_attrs: Optional[list[dict[str, str | int | float]]] = None

        # Map for fast vertex id lookups (osmid -> igraph id)
        self.vs_ids: Optional[dict[int, int]] = None

    def startDocument(self) -> None:
        """Init buffers."""
        self._es = []
        self._es_attrs = []
        self._vs_attrs = []
        self.vs_ids = dict()
    
    def startElement(self, element_type: str, attrs: xmlreader.AttributesImpl) -> None:
        """Parse XML elements."""

        if element_type == "node":
            osmid = int(attrs["id"])
            self._element = CoreElementBuffer(element_type, osmid)

            self._element.attrs["x"] = float(attrs["lon"])
            self._element.attrs["y"] = float(attrs["lat"])

            # Since igraph id is incremental int, we cen use length of the map
            self.vs_ids[osmid] = len(self._vs_attrs)
        
        elif element_type == "way":
            osmid = int(attrs["id"])
            self._element = CoreElementBuffer(element_type, osmid)
        
        # Do not parse relations
        elif element_type == "relation":
            self._element = None

        # Parse tags (ignore tags inside relations)
        elif element_type == "tag" and self._element is not None:
            tag = attrs["k"]
            if tag in TAGS_TO_KEEP[self._element.type]:
                # TODO: Cast to bool/float/int based on attr type
                self._element.attrs[tag] = attrs["v"]

        # Parse paths inside way elements
        elif element_type == "nd":
            node_osmid = int(attrs["ref"])
            self._element.path.append(node_osmid)

    def endElement(self, element_type: str) -> None:
        """Append core element to buffers."""

        if element_type == "node":
            # NOTE: It might be a good idea to pass osmid as name attribute
            self._vs_attrs.append(self._element.attrs)
        
        elif element_type == "way":
            path = self._element.path
            attrs = self._element.attrs

            is_one_way = "oneway" in attrs and attrs["oneway"] in ONE_WAY_PATH_OPTIONS
            is_reversed = "oneway" in attrs and attrs["oneway"] in REVERSED_PATH_OPTIONS

            # Overwrite OSM oneway value with boolean
            attrs["oneway"] = is_one_way

            if is_reversed:
                path.reverse()
            
            # Parse edges and append to buffer
            n_edges = len(path) - 1
            for idx in range(n_edges):
                u = path[idx]
                v = path[idx + 1]

                self._es.append((u, v))
                self._es_attrs.append(attrs)

                if not is_one_way:
                    self._es.append((v, u))
                    self._es_attrs.append(attrs)

    def endDocument(self) -> None:
        """Finalize XML parsing and create graph."""

        self.g = ig.Graph(directed=True)

        metadata = {} # TODO: Add graph metadata
        for name, value in metadata.items():
            self.g[name] = value

        # Create graph from buffered data

        # Add vertices and attach attributes to them
        self.g.add_vertices(len(self._vs_attrs))
        for attrs, vertex in zip(self._vs_attrs, self.g.vs):
            vertex.update_attributes(attrs)
        
        self._vs_attrs = None  # drop vertex attributes buffer

        # Convert edges vertices from osmids to igraph ids
        edges_by_igraph_ids = [
            (
                self.vs_ids[u_name],
                self.vs_ids[v_name],
            )
            for u_name, v_name in self._es
        ]

        self._es = None  # drop edges buffer

        self.g.add_edges(edges_by_igraph_ids)
        for attrs, edge in zip(self._es_attrs, self.g.es):
            edge.update_attributes(attrs)
        
        self._es_attrs = None  # drop edges attributes buffer


# Test

In [21]:
DATA_PATH = "data/slovakia_borders.osm"

In [22]:
def parse_to_igraph(file_path: str) -> ig.Graph:
    parser = OSMParser()
    xml.sax.parse(file_path, parser)
    return parser.g, parser.vs_ids

g, vs_ids_map = parse_to_igraph(DATA_PATH)
print("Number of nodes:", len(g.vs))
print("Number of edges:", len(g.es))

Number of nodes: 84368
Number of edges: 168734


In [23]:
node_osmid = 4240267510
g.vs[vs_ids_map[node_osmid]]

igraph.Vertex(<igraph.Graph object at 0x7f82438acd40>, 84367, {'x': 22.5455842, 'y': 49.092004})

In [24]:
%timeit parse_to_igraph(DATA_PATH)

715 ms ± 52 ms per loop (mean ± std. dev. of 7 runs, 1 loop each)
